Let's build a transformer based, character level language model.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
with open(r'/content/drive/MyDrive/makemore/GPTFromScratch/input.txt','r',encoding='utf-8') as f:
  text = f.read()
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [3]:
chars = sorted(list(set(''.join(text))))
vocab_size = len(chars)
vocab_size,''.join(chars)

(65, "\n !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz")

In [4]:
# tokenize: convert the raw text as string to some sequence of integers
stoi = {s:i for i,s in enumerate(chars)}
itos = {i:s for i,s in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode('hi there'))
print(decode(encode('hi there')))

[46, 47, 1, 58, 46, 43, 56, 43]
hi there


In [5]:
data = torch.tensor(encode(text),dtype=torch.long)
print(data.shape,data.dtype)
print(data[:100])
# now the data is a single, long sequence of integers
# they are the encodings of the raw text

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [6]:
# train, validation split
n = int(len(data)*0.9)
train_data = data[:n]
val_data = data[n:]

In [7]:
block_size = 8
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f'when input is {context}, the target: {target}')
# so a single sample contains several (block_size) examples

when input is tensor([18]), the target: 47
when input is tensor([18, 47]), the target: 56
when input is tensor([18, 47, 56]), the target: 57
when input is tensor([18, 47, 56, 57]), the target: 58
when input is tensor([18, 47, 56, 57, 58]), the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]), the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]), the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]), the target: 58


In [8]:
torch.manual_seed(42)
batch_size = 4

def get_batch(split):
  # generate a small batch of data of inputs x and targets y
  data = train_data if split=='train' else val_data
  ix = torch.randint(0,len(data)-block_size,(batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+1+block_size] for i in ix])
  return x,y

xb,yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('------')
for b in range(batch_size):
  for t in range(block_size):
    context = xb[b,:t+1]
    target = yb[b,t]
    print(f'when input is {context.tolist()} the target: {target}')
# the input and the output are both sequences

inputs:
torch.Size([4, 8])
tensor([[57,  1, 46, 47, 57,  1, 50, 53],
        [ 1, 58, 46, 43, 56, 43,  1, 41],
        [17, 26, 15, 17, 10,  0, 32, 53],
        [57, 58,  6,  1, 61, 47, 58, 46]])
targets:
torch.Size([4, 8])
tensor([[ 1, 46, 47, 57,  1, 50, 53, 60],
        [58, 46, 43, 56, 43,  1, 41, 39],
        [26, 15, 17, 10,  0, 32, 53,  1],
        [58,  6,  1, 61, 47, 58, 46,  0]])
------
when input is [57] the target: 1
when input is [57, 1] the target: 46
when input is [57, 1, 46] the target: 47
when input is [57, 1, 46, 47] the target: 57
when input is [57, 1, 46, 47, 57] the target: 1
when input is [57, 1, 46, 47, 57, 1] the target: 50
when input is [57, 1, 46, 47, 57, 1, 50] the target: 53
when input is [57, 1, 46, 47, 57, 1, 50, 53] the target: 60
when input is [1] the target: 58
when input is [1, 58] the target: 46
when input is [1, 58, 46] the target: 43
when input is [1, 58, 46, 43] the target: 56
when input is [1, 58, 46, 43, 56] the target: 43
when input is [1, 58, 4

In [9]:
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
  def __init__(self,vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)
    # link of nn.embedding: https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html
    # note: this lookup table is trainable, and should be trained

  def forward(self,idx,targets=None):
    # idx and targets are both (B,T) tensor of integers
    # NOTE: both B and T here is not fixed, even though during training phase we use T=block_size, during generating phase T can be any integer number
    logits = self.token_embedding_table(idx) # (B,T,C)
    # loss = F.cross_entropy(logits,targets) # (B,T)
    # note: the shape of logits and the shape of target don't match, so this won't work
    # cross_entropy function asks the Channel dim to be the second dim
    # link of F.cross_entropy: https://pytorch.org/docs/stable/generated/torch.nn.functional.cross_entropy.html
    if targets == None: # this is for the generating phase, where we don't provide the targets
      loss = None
    else:
      # B, T, C = logits.shape
      # l = logits.view(B*T,C)
      # targets = targets.view(B*T)
      # Honestly, what Andrej did here is hard for me to understand, because I can't understand why we can just multiply the batch dim and the time dim
      # So I transpose the dim to make it more acceptable to me, and of course this will get the same answer
      l = torch.transpose(logits,1,2)
      loss = F.cross_entropy(l,targets)
    return logits,loss

  def generate(self,idx,max_new_tokens):
    # idx is (B,T) array of indices in the current context
    for _ in range(max_new_tokens):
      # get the predictions
      logits, loss = self(idx) # NOTE: this means that during this for loop, the model sees all the previous input to make the next prediction
      # focus only on the last time step
      logits = logits[:,-1,:] # (B,C)
      # apply softmax to get probabilities
      probs = F.softmax(logits,dim=-1) # (B,C)
      # sample from the distribution
      idx_next = torch.multinomial(probs,num_samples=1,replacement=True)
      # append sampled index to the running sequence
      idx = torch.cat((idx,idx_next),dim=1) # (B,T+1)
    return idx

m = BigramLanguageModel(vocab_size)
logits,loss = m(xb,yb)
print(logits.shape) # [batch_size, block_size, vocab_size]
print(yb.shape) # [batch_size, block_size]
print(loss)

idx = torch.zeros((1,1),dtype=torch.long) # (1,1) means there is a single batch, and the length of the input sequence is 0
print(decode(m.generate(idx,max_new_tokens=100)[0].tolist()))

torch.Size([4, 8, 65])
torch.Size([4, 8])
tensor(4.7013, grad_fn=<NllLoss2DBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [10]:
# now let's train the model
# create a pytorch optimizer
optimizer = torch.optim.AdamW(m.parameters(),lr=1e-3)
batch_size = 32
for steps in range(10000):
  # sample a batch of data
  xb,yb = get_batch('train')

  # evaluate the loss
  logits,loss = m(xb,yb)
  optimizer.zero_grad(set_to_none=True) # link: https://pytorch.org/docs/stable/generated/torch.optim.Optimizer.zero_grad.html
  loss.backward()
  optimizer.step()

print(loss.item())

2.572751045227051


In [11]:
idx = torch.zeros((1,1),dtype=torch.long)
print(decode(m.generate(idx,max_new_tokens=100)[0].tolist()))
# well, it's still pretty bad, but makes more sense than the untrained model
# after all we cannot expect much from this simple bigram model :(


Iyoteng h hasbe pave pirance
Rie hicomyonthar's
Plinseard ith henoure wounonthioneir thondy, y helti


# The mathematical trick in self-attention

In [30]:
# consider the following toy example
torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [31]:
# we want tokens in the sequence to communicate with each other
# in this case, we want the token to communicate with all the previous tokens
# let's use this simple example where the current token is the mean of all the previous tokens and itself
# i.e. we want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
  for t in range(T):
    xprev = x[b,:t+1] # (t,C)
    xbow[b,t] = torch.mean(xprev,0)

In [13]:
x[0] # the first batch

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [16]:
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [17]:
# this code does what we want, but it's very inefficient with two for loops
# let's see how we can do it using matrix multiplication
torch.manual_seed(42)
a = torch.ones(3,3)
b = torch.randint(0,10,(3,2)).float()
c = a@b
print('a=')
print(a)
print('b=')
print(b)
print('c=')
print(c)
# now, c is the 'time-wise' sum of b

a=
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c=
tensor([[14., 16.],
        [14., 16.],
        [14., 16.]])


In [18]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3)) # link: https://pytorch.org/docs/stable/generated/torch.tril.html
b = torch.randint(0,10,(3,2)).float()
c = a@b
print('a=')
print(a)
print('b=')
print(b)
print('c=')
print(c)
# with this low triagnle matrix a, c is the 'time-wise' sum of previous elements of b

a=
tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c=
tensor([[ 2.,  7.],
        [ 8., 11.],
        [14., 16.]])


In [24]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
a = a / a.sum(dim=1,keepdim=True)   #
b = torch.randint(0,10,(3,2)).float()
c = a@b
print('a=')
print(a)
print('b=')
print(b)
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [38]:
# we can generalize this, add the batch size
wei = torch.tril(torch.ones(T,T))
wei = wei / wei.sum(1,keepdim=True)
xbow2 = wei @ x # wei now is broadcasting as (B,T,T), and x is (B,T,C), this multiplication will give us (B,T,C)
xbow[0],xbow2[0]
# note: here we assume that the communication among time steps is taking the average, however in transformer we won't use such a simple 'communication'
# therefore, wei won't be torch.tril(torch.ones(T,T))

(tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]),
 tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]))

In [40]:
# another version, which is what we will use
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril==0,float('-inf')) # link: https://pytorch.org/docs/stable/generated/torch.Tensor.masked_fill_.html#torch.Tensor.masked_fill_
print(wei)
wei = F.softmax(wei,dim=-1)
print(wei)
xbow3 = wei @ x
xbow3[0]
# we can interprete wei as: how much of every time steps we want to integrate into our current step
# and the -inf means that the current time step cannot communicate to the future

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])


tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [44]:
# let's modify the model
# this model only adds a positional encoding and a linear layer after token embedding
n_embd = 32
device = 'cuda' if torch.cuda.is_available() else 'cpu'
class BigramLanguageModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size,n_embd) # embedding layer
    self.position_embedding_table = nn.Embedding(block_size,n_embd) # positional encoding
    self.lm_head = nn.Linear(n_embd,vocab_size)         # a linear layer

  def forward(self,idx,targets=None):
    B,T = idx.shape
    tok_emb = self.token_embedding_table(idx) # (B,T,n_embd)
    pos_emb = self.position_embedding_table(torch.arange(T,device=device)) # (T,n_embd)
    x = tok_emb + pos_emb # (B,T,n_embd)
    logits = self.lm_head(x) # (B,T,vocab_size)
    if targets == None:
      loss = None
    else:
      l = torch.transpose(logits,1,2)
      loss = F.cross_entropy(l,targets)

    return logits,loss
  def generate(self,idx,max_new_tokens):
    for _ in range(max_new_tokens):
      logits,loss = self(idx)
      logits = logits[:,-1,:]
      probs = F.softmax(logits,-1)
      idx_next = torch.multinomial(probs,num_samples=1,replacement=True)
      idx = torch.cat([idx,idx_next],dim=1)
    return idx

m = BigramLanguageModel()
context = torch.zeros((1,1),dtype=torch.long)
print(decode(m.generate(context,100)[0].tolist()))

# this will cause an error
# we will fix it later

IndexError: index out of range in self

In [54]:
# self-attention
torch.manual_seed(1337)
B,T,C = 4,8,32
x = torch.randn(B,T,C)

# let's see a single head perform self-attention
head_size = 16
key = nn.Linear(C,head_size,bias=False)
query = nn.Linear(C,head_size,bias=False)
value = nn.Linear(C,head_size)
k = key(x)  # B,T,head_size
q = query(x) # B,T,head_size
v = value(x) # B,T,head_size

wei = q @ k.transpose(-2,-1) * head_size**-0.5 # (B,T,16) @ (B,16,T) ---> (B,T,T)
tril = torch.tril(torch.ones(T,T))
wei = wei.masked_fill(tril==0,float('-inf'))
wei = F.softmax(wei,-1)
out = wei @ v
out.shape

torch.Size([4, 8, 16])

Notes:
  * Attention is a communication mechanism. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
  * There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
  * Each examples across batch dimensions is of course processed completely independently and never 'talk' to each other.
  * In an 'encoder' attention block, just delete the single line that does masking with tril, allowing all tokens to communicate. The block here is called a 'decoder' attention block because it has triangular masking, and is usually used in autoregressive settings.
  * 'self-attention' just means that the keys and values are produced from the same sources as queries, In 'cross-attention', the get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
  * 'scaled' attention additional diveds wei by 1/sqrt(head_size). This makes it so when input Q,K are unit, variance, wei will be unit variance too and softmax will stay diffuse and not saturate too much.

In [55]:
# here is the illustration about the 'scaled'
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-1,-2)
k.var(),q.var(),wei.var()

(tensor(1.0580), tensor(1.0693), tensor(17.7734))

In [58]:
wei = q @ k.transpose(-1,-2) * head_size**-0.5
wei.var()
# we can see that the variance is roughly preserved

tensor(1.1108)

In [59]:
# why we want to preserve the variance
torch.softmax(torch.tensor([0.1,-0.2,0.3,-0.2,0.5]),dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [61]:
torch.softmax(torch.tensor([0.1,-0.2,0.3,-0.2,0.5])*8,dim=-1)
# we can see that the output of softmax 'sharpens' to the large number

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [62]:
class Head(nn.Module):
    def __init__(self):
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size))) # link: https://pytorch.org/docs/stable/generated/torch.nn.Module.html

    def forward(self, x):
        B, T, C = x.shape
        q = self.query(x) # B,T,head_size
        k = self.key(x)
        v = self.value(x)
        wei = q @ k.transpose(-1, -2) * head_size ** -0.5 # B,T,T
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        out = wei @ v # B,T,head_size
        return out

In [ ]:
# now let's add a head to the language model
class BigramLanguageModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size,n_embd) # embedding layer
    self.position_embedding_table = nn.Embedding(block_size,n_embd) # positional encoding
    self.sa_head = Head(n_embd)                 # self-attention
    self.lm_head = nn.Linear(n_embd,vocab_size)         # a linear layer

  def forward(self,idx,targets=None):
    B,T = idx.shape
    tok_emb = self.token_embedding_table(idx) # (B,T,n_embd)
    pos_emb = self.position_embedding_table(torch.arange(T,device=device)) # (T,n_embd)
    x = tok_emb + pos_emb # (B,T,n_embd)
    x = self.sa_head(x) # apply one head of self-attention, (B,T,C)
    logits = self.lm_head(x) # (B,T,vocab_size)
    if targets == None:
      loss = None
    else:
      l = torch.transpose(logits,1,2)
      loss = F.cross_entropy(l,targets)

    return logits,loss
  def generate(self,idx,max_new_tokens):
    for _ in range(max_new_tokens):
      logits,loss = self(idx[:,-block_size])    # the length of the sequence should not be bigger than block_size
      logits = logits[:,-1,:]
      probs = F.softmax(logits,-1)
      idx_next = torch.multinomial(probs,num_samples=1,replacement=True)
      idx = torch.cat([idx,idx_next],dim=1)
    return idx